# Discontinuous Galerkin for 1D Convection-Diffusion Equation

We consider the general equation
$$\partial_t \phi + \underbrace{\nabla(\beta \phi)}_{\text{convection}} + 
\underbrace{\nabla(\gamma \nabla \phi)}_{\text{diffusion}} = 0$$
Multiplying through by a test function $\psi$ and integrating the first and third terms by parts gives:
$$\partial_t \int_{\Omega} \phi\psi + \int_{\Omega} \nabla (\beta \phi) \psi - \int_{\Omega}(\gamma \nabla \phi)\nabla \psi + \int_{\partial \Omega}\gamma \nabla \phi \psi = 0$$
The Discontinous Galerkin formulation is then
$$\sum_T\int_T \partial_t (\phi \psi) + \sum_T\int_T ( \nabla \beta \phi) \psi + \sum_T\int_{\partial T} \beta_n (\hat{\phi}-\phi)\psi - \sum_T\int_T \nabla(\gamma \phi)\nabla \psi$$ 
$$ - \frac{1}{2} \sum_F\int_F \lbrace n\nabla\phi\rbrace [\phi] - \frac{1}{2}\sum_F\int_F\lbrace n \nabla\phi \rbrace [\psi] + \frac{\alpha p^2}{h} \sum_F \int_F [\phi][\psi]$$
In the case when $\beta$ and $\gamma$ are constants and the dimension of $\Omega$ is 1, we get:
$$\sum_T\int_T \dot{(\phi \psi)} + \beta \sum_T\int_T \phi' \psi + \beta \sum_T\int_{\partial T} (\hat{\phi}-\phi)\psi - \gamma \sum_T\int_T \phi' \psi'$$ 
$$ - \frac{\gamma}{2} \sum_F\int_F \lbrace n\nabla\phi\rbrace [\phi] - \frac{\gamma}{2}\sum_F\int_F\lbrace n \nabla\phi \rbrace [\psi] + \frac{\gamma \alpha p^2}{h} \sum_F \int_F [\phi][\psi]$$

In [1]:
import netgen.gui
%gui tk
from mesh_util import uniform_1d_mesh
from ngsolve import *
import ngsolve.internal as ngint
ngint.viewoptions.drawedges = 1

mesh = Mesh(uniform_1d_mesh())


The space is responsible for allocating the matrix graph. Tell it that it should reserve entries for the coupling terms:

In [4]:
order=4
fes = L2(mesh, order=order, dgjumps = True)
u,v = fes.TnT()


Every facet has a master element. The value from the other element is referred to via the Other() operator:

In [5]:
jump_u = u-u.Other()
jump_v = v-v.Other()
n = specialcf.normal(1)
mean_dudn = 0.5*n * (grad(u)+grad(u.Other()))
mean_dvdn = 0.5*n * (grad(v)+grad(v.Other()))

Integrals on facets are computed by setting skeleton=True. This iterates over all internal facets. Additionally setting BND iterates only over boundary facets:


In [7]:
alpha = 4
h = specialcf.mesh_size
acd = BilinearForm(fes)
acd += SymbolicBFI(grad(u)*grad(v))
acd += SymbolicBFI(alpha*order**2/h*jump_u*jump_v, skeleton=True)
acd += SymbolicBFI(alpha*order**2/h*u*v, BND, skeleton=True)
acd += SymbolicBFI(-mean_dudn*jump_v -mean_dvdn*jump_u, skeleton=True)
acd += SymbolicBFI(-n*grad(u)*v-n*grad(v)*u, BND, skeleton=True)


The IfPos checks whether the first argument is positive. Then it returns the second one, else the third one. This is used to define the upwind flux. The check is performed in every integration-point on the skeleton:

In [8]:
b = CoefficientFunction( 20 )                     # scalar
acd += SymbolicBFI(-b * u * grad(v))              # term 1 in second equation above
uup = IfPos(b*n, u, u.Other())                    # u_upwind (upwind flux)
acd += SymbolicBFI(b*n*uup*jump_v, skeleton=True) # term 2 in second equation above

In [9]:
acd.Assemble()

In [11]:
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()

In [12]:
gfu = GridFunction(fes)
gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
Draw (gfu)

## Explicit time stepping with a DG formulation

Explicit Euler:

$$\sum_T \int_T \phi^{n+1}\psi = \sum_T \int_T \phi^{n}\psi$$ 
$$ - \Delta t \left\lbrace \sum_T \int_T \beta \cdot \nabla \phi \psi + \int_{\partial T} \beta_n (\hat{\phi} - \phi)\psi + \int_T \nabla(\gamma \phi)\nabla \psi + \frac{1}{2} \sum_F\int_F \lbrace n\nabla\phi\rbrace [\phi] + \frac{1}{2}\int_F\lbrace n \nabla\phi \rbrace [\psi] - \frac{\alpha p^2}{h} \int_F [\phi][\psi] \right\rbrace, \hspace{2em} \forall \psi \in V_h$$

$$M \phi^{n+1} = M \phi^n -\Delta t C \phi^n$$

Letting $u\equiv\phi$ and $v\equiv\psi$
$$\sum_T \int_T u^{n+1}v = \sum_T \int_T u^{n}v$$ 
$$ - \Delta t \left\lbrace \sum_T \int_T \beta \cdot \nabla u v + \int_{\partial T} \beta_n (\hat{u} - u)v + \int_T \nabla(\gamma u)\nabla v + \frac{1}{2} \sum_F\int_F \lbrace n\nabla u \rbrace [u] + \frac{1}{2}\int_F\lbrace n \nabla u \rbrace [v] - \frac{\alpha p^2}{h} \int_F [u][v] \right\rbrace, \hspace{2em} \forall v \in V_h$$

$$M u^{n+1} = M u^n -\Delta t C u^n$$

In our first example we set $u_0=f=0$


### Computing convection applications $Cu^n$

We compute $Cu^n$ and define the corresponding bilinear form $C$

. Some remarks on that:

- We can define the bilinear form without setting up a matrix. The operator application is the only thing we want to use from the bilinear form
- We require integrals on element boundaries. We use the keyword "element_boundary" to obtain quadrature rules for this.
- To distinguish inflow from outflow we take compute $b\cdot n$ Here the normal $n$ is available as a specialcf which has meaningful values on the skeleton, element boundaries or domain boundaries.
- To make cases with coefficient functions we use the IfPos CoefficientFunction which has three arguments:

   1. CF argument that decides on the evaluation. If expression is positive evaluate 2. otherwise 3.
   2. CF that is evaluated in the positive case
   3. CF that is evaluated in the negative case

To access the neighbor variables we can use **u.Other()**
To incorporate boundary conditions ($\hat{u}$ on inflow boundaries), we set up a BilinearForm which is technically not bilinear, but only affine linear in the first argument. For operator application this behaves as desired, for setting up matrices this does not make sense!